## Implementation

This script implements a real-time facial emotion recognition application using a pre-trained deep learning model.
It captures video from the default camera (webcam) and detects faces in the frames using the Haar Cascade classifier.
For each detected face, the script predicts the corresponding emotion (e.g., Angry, Happy) based on a pre-trained model.

In this code, a class FacialExpressionModel is defined for predicting facial expressions using a loaded neural network model. It has methods for initialization and emotion prediction. The class attribute EMOTIONS_LIST holds the emotion labels corresponding to the model's output classes. The init method loads the model architecture and weights from provided files. The predict_emotion method takes an input image, makes predictions using the loaded model, and returns the predicted emotion label. 

In [1]:
# Import Necessary Libraries
import cv2
import numpy as np
from keras.models import model_from_json
import os 
os.chdir('/Users/clementine/Desktop/Maths Project')

2023-08-11 23:34:14.361498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define a class for facial expression prediction using a loaded neural network model
class FacialExpressionModel(object):
    # List of emotion labels corresponding to model output classes
    EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

    # Initialize the class with a trained model's JSON and weights files
    def __init__(self, model_json_file, model_weights_file):
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)
            
        # Load the model weights from the weights file
        self.loaded_model.load_weights(model_weights_file)
        print("Model loaded from disk")
        # Display a summary of the loaded model architecture
        self.loaded_model.summary()
        
     # Predict emotion label for an input image
    def predict_emotion(self, img):
        # Make predictions using the loaded model
        self.preds = self.loaded_model.predict(img)
        
         # Return the emotion label with the highest predicted probability
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]

Below code forms a real-time facial emotion recognition application using computer vision techniques and a pre-trained deep learning model. By accessing the camera feed, the application captures video frames, detects faces, and predicts the emotions depicted by individuals in real time. It overlays the predicted emotion labels on detected faces and outlines the faces with rectangles. The code showcases the integration of a loaded emotion prediction model, Haar Cascade face detection, and OpenCV visualization to create an interactive and informative tool for recognizing emotions from live video streams.

In [3]:
# Initialize the camera capture object
cap = cap = cv2.VideoCapture(0)
# Load the Haar Cascade classifier for detecting faces
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Define the font style for text overlay on the video frames
font = cv2.FONT_HERSHEY_SIMPLEX

# Function to capture video frames, detect faces, and return necessary data
def getdata():
    _, fr = cap.read() # Read a frame from the camera
    gray = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale
    faces = faceCascade.detectMultiScale(gray, 1.3, 5)  # Detect faces using the Haar Cascade classifier
    return faces, fr, gray

# Function to start the facial emotion recognition application
def start_app(cnn):
    while cap.isOpened(): 
        faces, fr, gray_fr = getdata() # Capture data from the camera
        for (x, y, w, h) in faces: 
            fc = gray_fr[y:y + h, x:x + w] # Extract the face region
            roi = cv2.resize(fc, (48, 48)) # Resize the face region for prediction
            pred = cnn.predict_emotion(roi[np.newaxis, :, :, np.newaxis]) # Predict the emotion
            
            # Overlay the predicted emotion label and draw a rectangle around the face
            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 1)
            cv2.rectangle(fr, (x, y), (x + w, y + h), (255, 0, 0), 1)
        
        # Exit the application if the 'Esc' key is pressed
        if cv2.waitKey(1) == 27:
            break
        cv2.imshow('Facial Emotion Recognition', fr) # Display the frame with overlays
        
    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Main execution
if __name__ == '__main__': 
    model = FacialExpressionModel("model.json", "model.h5") # Load the emotion prediction model
    start_app(model)  # Start the real-time emotion recognition application

Model loaded from disk
Model: "DCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1664      
                                                                 
 block1_batchnorm1 (BatchNor  (None, 48, 48, 64)       256       
 malization)                                                     
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        102464    
                                                                 
 block1_batchnorm2 (BatchNor  (None, 48, 48, 64)       256       
 malization)                                                     
                                                                 
 block1_maxpool (MaxPooling2  (None, 24, 24, 64)       0         
 D)                                                              
                                       

1/1 [==============================] - 0s 28ms/step
